In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import math
sns.set(style="whitegrid")

In [49]:
df = pd.read_csv('F:/MyProjects/PraxisProjects/hospital-readmission-risk-prediction/notebooks/dataset/diabetic_data.csv', na_values=['?', '[]'], low_memory=False)
display(df.shape)
display(df.columns)
df.head()

(101766, 50)

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [50]:
df = df.drop_duplicates()
df.duplicated().sum()

np.int64(0)

In [51]:
df.isnull().sum().sum()

np.int64(374017)

In [52]:
def map_icd9(code):
    if pd.isna(code):
        return "Unknown"

    try:
        code = float(code)
    except:
        return "Unknown"

    if 250 <= code < 251:
        return "Diabetes"
    elif 390 <= code < 460:
        return "Circulatory"
    elif 460 <= code < 520:
        return "Respiratory"
    elif 520 <= code < 580:
        return "Digestive"
    elif 580 <= code < 630:
        return "Genitourinary"
    elif 140 <= code < 240:
        return "Cancer"
    elif 710 <= code < 740:
        return "Musculoskeletal"
    elif 800 <= code < 1000:
        return "Injury"
    else:
        return "Other"


In [53]:
for col in ["diag_1", "diag_2", "diag_3"]:
    df[col + "_group"] = df[col].apply(map_icd9)

In [54]:
df = df[df['gender'].isin(['Female', 'Male'])]
df['gender'].value_counts()

gender
Female    54708
Male      47055
Name: count, dtype: int64

In [55]:
age_mapping = {
    '[0-10)': 5,
    '[10-20)': 15,
    '[20-30)': 25,
    '[30-40)': 35,
    '[40-50)': 45,
    '[50-60)': 55,
    '[60-70)': 65,
    '[70-80)': 75,
    '[80-90)': 85,
    '[90-100)': 95
}

In [56]:
df['age'] = df['age'].map(age_mapping).astype(int)
display(df['age'].value_counts())

age
75    26066
65    22482
55    17256
85    17197
45     9685
35     3775
95     2793
25     1657
15      691
5       161
Name: count, dtype: int64

In [57]:
cols_to_group = ['admission_type_id', 'discharge_disposition_id', 'admission_source_id']

threshold = 0.01

for col in cols_to_group:
    value_counts = df[col].value_counts(normalize=True)
    
    keep_values = value_counts[value_counts >= threshold].index
    
    df[col] = df[col].astype(str)
    
    df[col] = df[col].where(df[col].isin(keep_values.astype(str)), 'Other')

for col in cols_to_group:
    print(df[col].value_counts())

admission_type_id
1        53988
3        18868
2        18480
6         5291
5         4785
Other      351
Name: count, dtype: int64
discharge_disposition_id
1        60232
3        13954
6        12902
Other     4038
18        3691
2         2128
22        1992
11        1642
5         1184
Name: count, dtype: int64
admission_source_id
7        57492
1        29564
17        6781
4         3187
6         2264
Other     1371
2         1104
Name: count, dtype: int64


In [58]:
df['total_visits'] = df['number_outpatient'] + df['number_emergency'] + df['number_inpatient']

print(df[['number_outpatient', 'number_emergency', 'number_inpatient', 'total_visits']].head())

   number_outpatient  number_emergency  number_inpatient  total_visits
0                  0                 0                 0             0
1                  0                 0                 0             0
2                  2                 0                 1             3
3                  0                 0                 0             0
4                  0                 0                 0             0


In [59]:
med_cols = [
    'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
    'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
    'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
    'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
    'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone'
]

df['num_med_changes'] = df[med_cols].isin(['Up', 'Down']).sum(axis=1)
 
df['num_med_active'] = (df[med_cols] != 'No').sum(axis=1)

insulin_map = {'No': 0, 'Steady': 1, 'Down': 2, 'Up': 3}
df['insulin_coded'] = df['insulin'].map(insulin_map)

print(df[['num_med_changes', 'num_med_active', 'insulin_coded']].head())

   num_med_changes  num_med_active  insulin_coded
0                0               0              0
1                1               1              3
2                0               1              0
3                1               1              3
4                0               2              1


In [60]:
df['num_med_active'] = (df[med_cols] != 'No').sum(axis=1)

df['interaction_visits_meds'] = df['total_visits'] * df['num_med_active']

print(df[['total_visits', 'num_med_active', 'interaction_visits_meds']].head())

   total_visits  num_med_active  interaction_visits_meds
0             0               0                        0
1             0               1                        0
2             3               1                        3
3             0               1                        0
4             0               2                        0


In [61]:
df.shape

(101763, 58)

In [62]:
df.isnull().sum().sum()

np.int64(374003)

In [63]:
df["readmitted_binary"] = df["readmitted"].map({
    "<30": 1,
    ">30": 0,
    "NO": 0
})

In [64]:
df.drop(columns=["readmitted"], inplace=True)
df["readmitted_binary"].value_counts(normalize=True)

readmitted_binary
0    0.888398
1    0.111602
Name: proportion, dtype: float64

In [65]:
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,diabetesMed,diag_1_group,diag_2_group,diag_3_group,total_visits,num_med_changes,num_med_active,insulin_coded,interaction_visits_meds,readmitted_binary
0,2278392,8222157,Caucasian,Female,5,NaN,6,Other,1,1,...,No,Diabetes,Unknown,Unknown,0,0,0,0,0,0
1,149190,55629189,Caucasian,Female,15,NaN,1,1,7,3,...,Yes,Other,Diabetes,Other,0,1,1,3,0,0
2,64410,86047875,AfricanAmerican,Female,25,NaN,1,1,7,2,...,Yes,Other,Diabetes,Unknown,3,0,1,0,3,0
3,500364,82442376,Caucasian,Male,35,NaN,1,1,7,2,...,Yes,Other,Diabetes,Circulatory,0,1,1,3,0,0
4,16680,42519267,Caucasian,Male,45,NaN,1,1,7,1,...,Yes,Cancer,Cancer,Diabetes,0,0,2,1,0,0


In [66]:
df.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'diag_1_group',
       'diag_2_group', 'diag_3_group', 'total_visits', 'num_med_changes',
       'num_med_

In [67]:
df.drop(columns=["encounter_id", "patient_nbr", "payer_code", "medical_specialty", "examide", "citoglipton", "weight", "max_glu_serum", "A1Cresult","diag_1", "diag_2", "diag_3", "number_outpatient", "num_procedures","num_med_active", "diag_2_group" , "tolbutamide", "tolazamide", "diag_3_group", "troglitazone", "race", "insulin", "glimepiride", "metformin-rosiglitazone", "pioglitazone", "repaglinide", "glipizide-metformin", "glyburide", "metformin-pioglitazone","acetohexamide", "glimepiride-pioglitazone", "rosiglitazone", "acarbose", "chlorpropamide", "glipizide", "miglitol", "glyburide-metformin", "nateglinide"], inplace=True)

In [68]:
df.shape

(101763, 20)

In [69]:
df.isnull().sum().sum()

np.int64(0)

In [70]:
df.head()

,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_medications,number_emergency,number_inpatient,number_diagnoses,metformin,change,diabetesMed,diag_1_group,total_visits,num_med_changes,insulin_coded,interaction_visits_meds,readmitted_binary
0,Female,5,6,Other,1,1,41,1,0,0,1,No,No,No,Diabetes,0,0,0,0,0
1,Female,15,1,1,7,3,59,18,0,0,9,No,Ch,Yes,Other,0,1,3,0,0
2,Female,25,1,1,7,2,11,13,0,1,6,No,No,Yes,Other,3,0,0,3,0
3,Male,35,1,1,7,2,44,16,0,0,7,No,Ch,Yes,Other,0,1,3,0,0
4,Male,45,1,1,7,1,51,8,0,0,5,No,Ch,Yes,Cancer,0,0,1,0,0


In [71]:
df.columns

Index(['gender', 'age', 'admission_type_id', 'discharge_disposition_id',
       'admission_source_id', 'time_in_hospital', 'num_lab_procedures',
       'num_medications', 'number_emergency', 'number_inpatient',
       'number_diagnoses', 'metformin', 'change', 'diabetesMed',
       'diag_1_group', 'total_visits', 'num_med_changes', 'insulin_coded',
       'interaction_visits_meds', 'readmitted_binary'],
      dtype='object')

In [72]:
df_subset = df[:10000]

In [73]:
df_subset["readmitted_binary"].value_counts(normalize=True)

readmitted_binary
0    0.8902
1    0.1098
Name: proportion, dtype: float64

In [74]:
df_subset.to_csv('F:/MyProjects/PraxisProjects/hospital-readmission-risk-prediction/notebooks/dataset/data_final10ktest.csv',index=False)